# 전처리 ( Preprocessing)

In [1]:
# -*- coding: utf-8 -*-

## Load package

In [7]:
import os
import pandas as pd 
from glob import glob

import sys
import re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np
from konlpy.tag import Twitter, Mecab
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 197504, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


## 사용자 사전 추가 및 Stopwords 를 위한 전처리 작업

### 언론사 목록 정리

> stopwords를 위해서

In [8]:
pressList = pd.read_excel('./data/presslist.xlsx')

with(open('./data/newspress.txt','w',encoding='utf-8')) as f:
    f.write('\n'.join(set(pressList['언론사'].tolist())))
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwordList = Stopwords('./data/newspress.txt')

### NIA Dictionary 파일의 태그 변환을 위한 Twitter 태그와 Hannanum(KAIST) 태그의 매칭파일 만들기

In [9]:
tagFile = pd.read_excel('./data/Korean_POS_tags_comparison.xlsx',sheet_name='chart3',dtype = str )

In [10]:
tagDict = dict()
for idx in tagFile.index:
    if tagFile.loc[idx]['Hannanum'][:3] =='XSN':
        tag = 'XSN'
    else:
        tag = tagFile.loc[idx]['Hannanum']
    tagDict[tag]=tagFile.loc[idx]['Twitter_Korean_Text']
    

In [11]:
def TagChange(tag):
    return tagDict[tag]

### NIA Dictionary파일로부터 Twitter 태그를 가진 파일 만들기

In [12]:
niaDictFile = pd.read_excel('./data/NIADic.xlsx')

niaDictFile = niaDictFile[niaDictFile['tag']!='xp']
niaDictFile.loc[:,'tag'] = niaDictFile['tag'].map(lambda x: TagChange(x.upper()))

for idx in niaDictFile.tag.unique():
    outfile = ('./data/'+'twitter_tagger_' + idx +'_TagChange_from_NIADic.txt')
    with open(outfile,'w',encoding='utf-8') as f:
        outdata = niaDictFile[niaDictFile['tag']==idx]['term']
        outdata.to_csv(outfile,index=False)

### 위키피디아 dump 파일로부터 title 정보만 추출하기

In [13]:
wikifile = './data/first_title_from_wikidumps.txt'
if not os.path.isfile(wikifile):
    outfile = open(wikifile,'w',encoding='utf-8')
    wikiTextList = glob('./data/wikidumps/text/A*')
    for folder in wikiTextList:
        fileList = glob(folder+'/*')
        for file in fileList:
            with open(file, 'r',encoding ='utf-8') as f:
                while 1:
                    line = f.readline()
                    if not line:break
                    try:
                        x1 = re.search('title',line).group()
                    except:
                        pass
                    else:
                        outfile.write(line)
    outfile.close()

### mecab과 customized-konlpy의 사용자사전 추가를 위한 namu wiki와 위키피디아 title 정보 전처리

In [14]:
outSet = set()
with open('./data/first_title_from_wikidumps.txt','r',encoding='utf-8') as f:
    while 1:
        line = f.readline()
        if not line:break
        try:
            tmp1 = re.split('title=',line)[1].strip()
        except:pass
        else:
            tmp2 = re.sub('["]','',tmp1)[:-1]
            tmp2 = html.unescape(tmp2).strip()
            outSet.add(tmp2)
    f.close()
arrayOut = pd.DataFrame(list(outSet))
outSet = set()
with open('./data/out_namu_wiki.txt','r',encoding='utf-8') as f:
    for line in f.readlines():
        line = html.unescape(line).strip()
        outSet.add(line)
    f.close()
arrayOut2 = pd.DataFrame(list(outSet))

df1 = pd.concat([arrayOut, arrayOut2])
print (len(df1))
df1.reset_index(drop=True, inplace = True)
df1.drop_duplicates(subset = [0], keep = False, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
df1 = pd.DataFrame(df1[0].str.split('/').str.get(0))
df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
df1 = pd.DataFrame(df1[0].str.split(':').str.get(0))
df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
df1 = pd.DataFrame(df1[0].str.split('\([a-zA-Z0-9가-힣\s\W\w\S]+\)').str.get(0))
df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop_duplicates(inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('[^a-zA-Z0-9가-힣\s]+$')].index, inplace = True)
df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('\([a-zA-Z]+\)')].index, inplace=True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('\([0-9]\)')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('\([가-힣\s]\)')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[0-9]+\.[0-9]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('[0-9]+\.[0-9]+[a-zA-Z]')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[0-9]+\.[0-9].[a-zA-Z]')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('[\.][0-9]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[\.][0-9]')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('[\.][a-zA-Z]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('^(\.[\.a-zA-Z0-9])+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('(.+png[.]+)')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('(.+jpg[.]+)')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('(.+\.+jpg+$)')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('(.+\.+png+$)')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('(.+\.+[a-zA-Z]+$)')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('[0-9]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[a-zA-Z]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[0-9a-zA-Z\./]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[\W]+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

1325438
1157924
885658
882163
882163
882135
882063
882017
881619
702748
616503


In [15]:
df1.drop(df1[df1[0].str.match('^[a-zA-Z ]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.\([a-zA-Z0-9가-힣\s]\)+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.\([^a-zA-Z0-9가-힣\s]\)+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('.[^a-zA-Z0-9가-힣\s]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[^a-zA-Z0-9가-힣\s]')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[^가-힣]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('^[0-9]+년')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.[0-9]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.[a-zA-Z]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.[\(]+[a-zA-Z]+[\)]')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('.[0-9]+.+[a-zA-Z]+$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.[~].')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[0-9]+[-]+[0-9]+')].index,inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('[0-9]+[-]+[0-9]')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('^[0-9 ]+\.+[0-9 ]+[a-zA-Z]+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.\([a-zA-Z0-9가-힣\s]+\)+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.drop(df1[df1[0].str.match('.+-[0-9]{4,}$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('[a-zA-Z0-9가-힣\s]+[0-9]{0,4}-[0-9]{1,2}$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.*[0-9]{0,4}-[0-9]{1,2}$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

600477
586771
577580
577397
577383
575476


In [16]:
df1.drop(df1[df1[0].str.match('.*-$')].index, inplace =True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('.+[0-9]{1,4}$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
df1.drop(df1[df1[0].str.match('^[0-9]{1,}[a-zA-Z]{1,}')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('.+II$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('.+I$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('[^a-zA-Z]+[\-a-zA-Z]{1,1}$')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('.+및.+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('.+의 .+')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('.+,[ ]{1,}')].index, inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1 = df1[df1[0].str.split(' ').str.len() < 5]
print (len(df1))
df1.drop(df1[df1[0] ==''].index, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('[a-zA-Z0-9가-힣\s]{1,1}[^a-zA-Z0-9가-힣][a-zA-Z0-9가-힣\s]{1,1}$')].index, inplace = True)
df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('.+imf.*')].index, inplace = True)
df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))

df1.drop(df1[df1[0].str.match('.+[~`\!@#\$%\^\&\*\(\)\_\+=\{\}\[\]\\\|;:\'\"<,>\?/].*')].index, inplace = True)

df1.dropna(inplace = True)
df1.reset_index(drop=True, inplace = True)
print (len(df1))
df1.to_csv('./data/second_title_from_wikidumps.txt',index=False, header = False, encoding='utf-8')

561700
561427
561322
559890
559197
531241
530612
524465
524464
523685
523684
520667


### NIADic와 NAMUWIKI& WIKIPEDIA의 TITLE로 부터 추출된 명사 데이터 정제후 합치기

In [35]:
def DataFilter(dataPath):
    print (dataPath)
    datalist = glob(dataPath+'*.txt')
    datalist.extend(glob(dataPath+'*.csv'))
    datalist.extend(glob(dataPath+'*.CSV'))
    datadf = pd.DataFrame(columns=[0])
    for data in datalist:
        df = pd.read_csv(data,encoding='utf-8', header=None)
        if len(datadf) == 0:
            datadf = df
        else:
            datadf = pd.concat([datadf, df])
    datadf = datadf[0]
    print (datadf.shape)

    datadf.drop_duplicates(inplace = True)
    datadf.reset_index(drop=True, inplace = True)
    datadf.dropna(inplace = True)
    datadf.reset_index(drop=True, inplace = True)
    print (datadf.shape)

    datadf = datadf.apply(lambda x: re.sub(' ','',x))
    print (datadf.shape)

    datadf.drop(datadf[datadf.str.contains('[a-zA-Z]+[양씨군사의는은이가히]+$')].index, inplace = True)
    datadf.reset_index(drop=True, inplace = True)
    print (datadf.shape)

    datadf.drop(datadf[datadf.str.contains('[a-zA-Z가-힣]+[양씨군사의는은이가히]+$')].index, inplace = True)
    datadf.reset_index(drop=True, inplace = True)
    print (datadf.shape)

    datadf.drop(datadf[datadf.str.contains('.+[씨는은이가]+$')].index, inplace=True)
    datadf.reset_index(drop=True, inplace = True)
    print (datadf.shape)

    eng = datadf[datadf.str.match('[a-zA-Z0-9]+$')]
    eng.reset_index(drop=True,inplace = True)
    datadf.drop(datadf[datadf.str.match('[a-zA-Z0-9]+$')].index, inplace=True)
    datadf.reset_index(drop=True, inplace = True)
    datadf.drop(datadf[datadf.str.match('[a-zA-Z0-9가-힣]+으로$')].index, inplace = True)
    datadf.reset_index(drop=True, inplace = True)
    datadf.dropna(inplace = True)
    datadf.drop_duplicates(inplace = True)
    datadf.reset_index(drop=True, inplace = True)
    print (datadf.shape)
    return datadf

In [37]:
datapath = './data/checkfile/a/'
datadf = DataFilter(datapath)
print (datadf.shape)
datadf.reset_index(drop=True, inplace = True)

./data/checkfile/a/
(1313179,)
(1135138,)
(1135138,)
(1135122,)
(1076509,)
(1076419,)
(1016015,)
(1016015,)


### Mecab & twitter를 활용한 품사 태깅으로 명사만 선별

In [45]:
datadf2 = datadf.apply(ct.pos)

In [46]:
datadf3 = datadf.apply(mecab.pos)

### 추가 제거 
* ex) 엄마를말하다, 폼은일시적이지만클래스는영원하다 같은 끝이 ~다로 끝나는 TITLE 제거

In [56]:
datadf2_1 = datadf2[datadf2.apply(lambda x: len(list(map(lambda y: y[1], x)))) == 1]
print (datadf2_1.shape)
datadf2_1 = datadf2_1[datadf2_1.apply(lambda x: x[0][1]=='Noun')]
datadf2_2 = datadf2_1[datadf2_1.apply(lambda x: x[0][1]!='Noun')]
print (datadf2_2.shape)
datadf2_2 = datadf2_2.apply(lambda x: x[0][0])
#datadf2_2.reset_index(drop=True, inplace = True)
print (datadf2_1.shape)
datadf2_1 = datadf2_1.apply(lambda x: x[0][0])
#datadf2_1.reset_index(drop=True, inplace = True)

(232843,)
(0,)
(230911,)


In [59]:
datadf3_1 = datadf3[datadf3.apply(lambda x: len(list(map(lambda y: y[1], x)))) == 1]
#datadf3_1.reset_index(drop=True, inplace = True)
print (datadf3_1.shape)
datadf3_2 = datadf3_1[datadf3_1.apply(lambda x: x[0][1] in ['NNG','NNP'])]
print (datadf3_2.shape)
datadf3_2 = datadf3_2.apply(lambda x: x[0][0])
#datadf3_2.reset_index(drop=True, inplace = True)

(318891,)
(313737,)


In [60]:
datadf3_3 = datadf3_1[datadf3_1.apply(lambda x: not x[0][1] in ['NNG','NNP'])]
print (datadf3_3.shape)
#datadf3_3.reset_index(drop= True, inplace = True)
datadf3_3 = datadf3_3.apply(lambda x: x[0][0])

(5154,)


In [127]:
datadf.drop(datadf[datadf3_3.index].index, inplace = True)
datadf.dropna(inplace = True)
datadf.drop_duplicates(inplace = True)
datadf.reset_index(drop=True, inplace = True)
print (datadf.shape)

(1010861,)


* 태깅이 하나만 된 것 중에서 명사가 아닌 것은 다 제외하기로 함

### 단어 추가
* 사전 획득 soynlp님 github page [github](https://github.com/lovit/sharing_korean_dictionary)

In [49]:
from collections import abc
import json

class Dictionary:
    def __init__(self, json_path):
        self._load_dictionary(json_path)
    
    def _load_dictionary(self, json_path):
        with open(json_path, encoding='utf-8') as f:
            json_object = json.load(f)
        self.meta = json_object.get('meta', {})
        self.hierarchy = json_object.get('dictionary', {})
        self.hierarchy = self._nested_dict_to_set(self.hierarchy)
        
    def _nested_dict_to_set(self, nested):
        return {key:set(value) if not isinstance(value, abc.Mapping) else self._nested_dict_to_set(value) for key, value in nested.items()}
    
    def show_hierarchy(self, nested=None, depth=0):
        if nested == None:
            nested = self.hierarchy
        for key, value in nested.items():
            print('{}|--{}'.format('   '*depth, key))
            if isinstance(value, abc.Mapping):
                self.show_hierarchy(value,depth+1)
    
    def get_words(self, categories=None):
        wordset = set()
        root = self.hierarchy
        if categories:
            if type(categories) == str:
                categories = categories.split()
            for category in categories:
                root = root.get(category, {})
        if not isinstance(root, dict):
            return set(root)
        for _, words in self._nested_dict_iter(root):
            wordset.update(words)
        return wordset
    
    def _nested_dict_iter(self, nested):
        for key, value in nested.items():
            if isinstance(value, abc.Mapping):
                yield from self._nested_dict_iter(value)
            else:
                yield key, value
    
    def find_categories(self, word):
        categories = set()
        for key, words in self._nested_dict_iter(self.hierarchy):
            if word in words:
                categories.add(key)
        return categories
    
    def __str__(self):
        return 'Dictionary:{}'.format(self.meta.get('name', ''))

In [133]:
x1 = pd.Series(list(Dictionary('./data/checkfile/dictionary_with_info_GICS.json').get_words()))
x2 = pd.Series(list(Dictionary('./data/checkfile/nouns_from_economy_news.json').get_words()))

In [154]:
x1 = x1[~x1.str.match('[a-zA-Z0-9\W]')]
x2 = x2[~x2.str.match('[a-zA-Z0-9\W]')]
wordsList = pd.concat([x1,x2])
wordsList.reset_index(drop=True, inplace = True)
wordsList.drop_duplicates(inplace = True)
wordsList.reset_index(drop=True, inplace = True)
wordsList.dropna(inplace = True)
print (wordsList.shape)


(128758,)


In [155]:
wsl2_ct = wordsList.apply(ct.pos)
wsl2_mecab = wordsList.apply(mecab.pos)

In [157]:
wsl2_ct.dropna(inplace = True)
print (wsl2_ct.shape)
wsl2_mecab.dropna(inplace = True)
print (wsl2_mecab.shape)

(128758,)
(128758,)


In [173]:
x = wsl2_ct[wsl2_ct.apply(lambda x: x[-1][-1] in ['Josa','Eomi', 'Suffix'])].index | wsl2_mecab[wsl2_mecab.apply(lambda x: x[-1][-1] in ['EC','EP','EF','ETM','ETN','JC','JKB','JKB',
 'JKC','JKG','JKO','JKQ','JKS','JKV','XSA','XSV'])].index

In [180]:
words1 = wordsList[x]
words2 = wordsList[pd.Int64Index(np.arange(len(wordsList))).difference(x)]

In [209]:
lstWordList = pd.concat([words2, words1[~words1.str.match('.+[써씨더서은만랑큼음는인에이게가으로기다을를들지측]$')]])
lstWordList.reset_index(drop=True, inplace = True)

In [211]:
print (lstWordList.shape)

(120635,)


In [212]:
lstOutcomeWords = pd.concat([datadf, lstWordList])
print (lstOutcomeWords.shape)
lstOutcomeWords.dropna(inplace=True)
lstOutcomeWords.reset_index(drop=True, inplace = True)
lstOutcomeWords.drop_duplicates(inplace = True)
lstOutcomeWords.reset_index(drop=True, inplace = True)
print (lstOutcomeWords.shape)

(1131496,)
(1091993,)


In [215]:
lstOutcome = lstOutcomeWords.apply(mecab.pos)

In [218]:
lstOutcome.to_csv('./data/checkfile/merged_file.txt',index=False, header = False, encoding='utf-8')

In [227]:
lstOutcome1 = lstOutcome[lstOutcome.apply(lambda x: len(list(map(lambda y: y[1], x)))) == 1]
lstOutcome1.reset_index(drop=True, inplace = True)
lstOutcome2 = lstOutcome[lstOutcome.apply(lambda x: len(list(map(lambda y: y[1], x)))) != 1]
lstOutcome2.reset_index(drop=True, inplace = True)

### mecab용 사용자사전 제작

In [228]:
def Jongsung(x):
    if (ord(x[-1])-0xAC00)%28 == 0:
        return 'F'
    else:
        return 'T'

In [229]:
a = lstOutcome1.apply(lambda x: x[0][0])
b = pd.Series()
#c = datadf3_1.apply(lambda x: '+'.join(list(map(lambda y: y[1], x))))
c =pd.Series(['NNP']*len(a))
d = pd.Series(['*']*len(a))
d1 = a.apply(lambda x: Jongsung(x))
x1 = pd.concat([a,b,b,b,c,d,d1,a,c,d,d,d],axis = 1)

In [230]:
a = lstOutcome2.apply(lambda x: ''.join(list(map(lambda y: y[0], x))))
b = pd.Series()
c = pd.Series(['NNP']*len(a))
d = pd.Series(['*']*len(a))
d1 = a.apply(lambda x: Jongsung(x))
e = pd.Series(['Compound']*len(a))
f = pd.Series(['*']*len(a))
g = pd.Series(['*']*len(a))
h = lstOutcome2.apply(lambda x: '+'.join(list(map(lambda y: y[0]+'/'+y[1]+'/*', x))))
x2 = pd.concat([a,b,b,b,c,d,d1,a,e,f,g,h],axis = 1)

ex) 
계몽전의,1,2,3,NNP,*,T,계몽전의,Compound,*,*,계몽/NNG/*+전의/NNG/*

In [233]:
x1.columns = list(range(len(x1.columns.values)))

In [234]:
x2.columns = list(range(len(x2.columns.values)))

In [235]:
m = pd.concat([x1,x2])
m.reset_index(drop = True, inplace = True)
m.to_csv('./data/checkfile/for_mecab_dic.csv',index=False, header = False, encoding='utf-8')

In [236]:
m.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,가온,NaN,NaN,NaN,NNP,*,T,가온,NNP,*,*,*
1,관우,NaN,NaN,NaN,NNP,*,F,관우,NNP,*,*,*
2,규,NaN,NaN,NaN,NNP,*,F,규,NNP,*,*,*
3,김준면,NaN,NaN,NaN,NNP,*,T,김준면,NNP,*,*,*
4,니노,NaN,NaN,NaN,NNP,*,F,니노,NNP,*,*,*
